In [1]:
import pandas as pd
import pickle
import re
import numpy as np
from tqdm import tqdm
from collections import defaultdict 
import pickle

In [2]:
import os
from os import listdir
import sys

In [3]:
file_dir = '../'

data_dir = file_dir + 'data/'
raw_data_dir = data_dir + 'raw/'
result_dir = data_dir + 'results/'

In [4]:
%run ../src/food_parser.py

[nltk_data] Downloading package words to /Users/joey-hou/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/joey-
[nltk_data]     hou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/joey-
[nltk_data]     hou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/joey-hou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# !pip install pyspellchecker
from spellchecker import SpellChecker

import re
import string
import nltk

from nltk.corpus import stopwords, words
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
wnl = WordNetLemmatizer()

nltk.download('averaged_perceptron_tagger')

# Load stop words
stop_words = stopwords.words('english')
# Updated by Katherine August 23rd 2020
stop_words.remove('out') # since pre work out is a valid beverage name
stop_words.remove('no')
stop_words.remove('not')

wordlist = words.words()

[nltk_data] Downloading package words to /Users/joey-hou/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/joey-
[nltk_data]     hou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/joey-hou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/joey-
[nltk_data]     hou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/joey-hou/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Data Formatting / Making test file

In [6]:
public_all_pickle_file = open(raw_data_dir + 'public_0924_basline_usable_expanded.pickle', 'rb')
public_all = pickle.load(public_all_pickle_file)
public_all.head(2)

,ID,unique_code,research_info_id,desc_text,food_type,original_logtime,original_logtime_notz,day,local_time,time,week_from_start,year
281,282,alqt15090005,150,Water,w,09-09-2015 8:20PM JST,2015-09-09 20:20:00,2015-09-09,20.333333,20:20:00,1,2015
282,283,alqt15090005,150,Soup,f,09-09-2015 10:38PM JST,2015-09-09 22:38:00,2015-09-09,22.633333,22:38:00,1,2015


In [7]:
test_data = public_all.sample(2000)[['desc_text', 'food_type']].copy().reset_index(drop = True)
test_data.shape

(2000, 2)

In [8]:
# test_data.to_csv(raw_data_dir + 'test_data.csv', index = False)

In [9]:
# public_all.reset_index(drop = True).to_csv(raw_data_dir + 'all_data.csv', index = False)

# Building improved food annotations

In [10]:
fp = FoodParser()

## Load known grams

In [11]:
known_grams = pd.read_csv('../resources/combined_gram_set.csv')
known_grams.head(2)

,gram_key,food_type,gram_type,tag1,tag2,tag3,tag4
0,chilli,f,1,NaN,NaN,NaN,NaN
1,chile,f,1,NaN,NaN,NaN,NaN


In [12]:
known_grams.tail(2)

,gram_key,food_type,gram_type,tag1,tag2,tag3,tag4
981,pre workout,b,2,preworkout,NaN,NaN,NaN
982,preworkout,b,1,NaN,NaN,NaN,NaN


In [13]:
known_grams_lst = list(known_grams.gram_key.values)
known_grams_set = set(known_grams_lst)

## Training Data Pre-processing

In [14]:
my_stop_words = ['oc', 'amp', 'homemade', 'food', 'foodporn', 'v', 'x', 'recipe', 'best', 'il', 'ever',
                'time', 'attempt', 'first', 'ate', 'made', 'home', 'today', 'make', 'friend', 'local',
                'new', 'day', 'birthday', 'like', 'amazing', 'de', 'happy', 'year', 'plate', 'video', 
                'cooked', 'dish', 'house', 'os', 'tried', 'super', 'perfect', 'way', 'delicious', 
                'night', 'last', 'eating', 'know', 
                 'mg', 'oz', 'and', 'w', 'x', 'n', 'ml'
                 'handful']

# Function for removing numbers
def handle_numbers(text):
    clean_text = text
    clean_text = re.sub('[0-9]+\.[0-9]+', '' , clean_text)
    clean_text = re.sub('[0-9]+', '' , clean_text)
    clean_text = re.sub('\s\s+', ' ', clean_text)
    return clean_text

# Function for removing punctuation
def drop_punc(text):
    clean_text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    return clean_text

# Remove normal stopwords
def remove_stop(my_text):
    text_list = my_text.split()
    return ' '.join([word for word in text_list if word not in stop_words])

def remove_my_stop(text):
    text_list = text.split()
    return ' '.join([word for word in text_list if word not in my_stop_words])

def remove_plural(w):
    return wnl.lemmatize(w)

def pre_processing(text):
    text = text.lower()
    text = remove_my_stop(remove_stop(handle_numbers(drop_punc(text)))).strip()
    return ' '.join([remove_plural(w) for w in text.split()])

In [15]:
all_text = public_all.desc_text.values

In [16]:
all_text_cleaned = [pre_processing(tx) for tx in tqdm(all_text)]

100%|██████████| 1653926/1653926 [00:50<00:00, 33062.26it/s]


In [17]:
len(all_text_cleaned)

1653926

In [18]:
all_text_cleaned[:3]

['water', 'soup', 'yogurt']

In [19]:
all_food_type = list(public_all.food_type.values)
all_food_type[:3]

['w', 'f', 'b']

In [20]:
training_texts = list(zip(all_text_cleaned, all_food_type))
training_texts[:3]

[('water', 'w'), ('soup', 'f'), ('yogurt', 'b')]

## Implement food mining

In [21]:
training_texts[:3]

[('water', 'w'), ('soup', 'f'), ('yogurt', 'b')]

In [22]:
list(known_grams_set)[:3]

['wholemeal', 'beef taco', 'chicken sausage']

In [23]:
def handle_single_phrase(ph, curr_phrase_dict, known_phrase_dict, known_sub_phrases_set):
    # Check known phrases list
    try:
        known_phrase_dict[ph] += 1
        return 0
    except:
        pass
    # Add to current phrase list if its not a subphrase
    if ph not in known_sub_phrases_set:
        curr_phrase_dict[ph] += 1
        return 1
    return 0

def prepare_phrase_base(known_grams_set):
#     known_grams_lst = list(known_grams_set)
#     known_sub_phrases_lst = []
#     tokenlized_known_phrases = [nltk.word_tokenize(ph) for ph in known_grams_lst]
#     for ph_length in [1, 2, 3, 4, 5]:
#         for tokens in tokenlized_known_phrases:
#             if len(tokens) < ph_length:
#                 continue
#             for tu in nltk.ngrams(tokens, ph_length):
#                 known_sub_phrases_lst.append(' '.join(tu))
#     known_sub_phrases_set = set(known_sub_phrases_lst)

    all_known_phrases_lst = []
    for file in ['quad_grams.csv', 'bi_grams.csv', 'uni_grams.csv', 'tri_grams.csv', 'penta_grams.csv']:
        curr_df = pd.read_csv('../resources/' + file)
        curr_phrases = curr_df.key.values
        all_known_phrases_lst += list(curr_phrases)
    all_known_sub_phrases_set = set(all_known_phrases_lst)
    return all_known_sub_phrases_set

def run_training(known_grams_set, training_texts):
    '''
    Input format:
        - known_grams_set: set object, e.g. {'water', 'preworkout', ...}
        - training_texts: list object, e.g. [('water', 'w'), ('preworkout', 'b'), ...]
    '''
    
    # Make a spellChecker instance
    # spell = SpellChecker()

    # Known phrases 
    known_phrase_dict = dict(zip(known_grams_set, [0] * len(list(known_grams_set))))
    known_sub_phrases_set = prepare_phrase_base(known_grams_set)
    
    # Scanning through the documents
    curr_phrase_dict = defaultdict(lambda : 0)

    for doc_pair in tqdm(training_texts):
        doc, food_type = doc_pair
        tokens = nltk.word_tokenize(doc)
        
        # Uni-grams
        [handle_single_phrase(ph, curr_phrase_dict, known_phrase_dict, known_sub_phrases_set) for ph in tokens]
        
        # N-grams
        [handle_single_phrase(' '.join(list(ph)), curr_phrase_dict, known_phrase_dict, known_sub_phrases_set) for ph in nltk.ngrams(tokens, 2)]
        [handle_single_phrase(' '.join(list(ph)), curr_phrase_dict, known_phrase_dict, known_sub_phrases_set) for ph in nltk.ngrams(tokens, 3)]
        [handle_single_phrase(' '.join(list(ph)), curr_phrase_dict, known_phrase_dict, known_sub_phrases_set) for ph in nltk.ngrams(tokens, 4)]
        [handle_single_phrase(' '.join(list(ph)), curr_phrase_dict, known_phrase_dict, known_sub_phrases_set) for ph in nltk.ngrams(tokens, 5)]
        
    return dict(known_phrase_dict), dict(curr_phrase_dict)

In [24]:
known_phrase_dict, curr_phrase_dict = run_training(known_grams_set, training_texts)

100%|██████████| 1653926/1653926 [02:38<00:00, 10438.05it/s]


In [25]:
curr_phrase_dict = dict(curr_phrase_dict)

In [45]:
sorted_items = sorted(curr_phrase_dict.items(), key = lambda x: x[1], reverse = True)
sorted_items = [pair for pair in sorted_items if pair[1] > 200]

In [46]:
len(sorted_items)

1731

In [47]:
sorted_items[:40]

[('ml', 12170),
 ('herbal', 8492),
 ('b', 7861),
 ('herbal tea', 7603),
 ('g', 7277),
 ('free', 7261),
 ('tea milk', 6322),
 ('no', 5634),
 ('med', 4819),
 ('cider', 4772),
 ('stevia', 4157),
 ('glass water', 4075),
 ('vitamin c', 4008),
 ('bannana', 3887),
 ('unsweetened', 3612),
 ('bulletproof', 3416),
 ('apple cider', 3360),
 ('crisp', 3353),
 ('kefir', 2986),
 ('bulletproof coffee', 2901),
 ('tsp', 2878),
 ('coffe', 2867),
 ('cider vinegar', 2866),
 ('keto', 2866),
 ('broth', 2839),
 ('no sugar', 2812),
 ('heavy', 2792),
 ('mcg', 2771),
 ('soy milk', 2698),
 ('stick', 2686),
 ('ml water', 2686),
 ('pumpkin seed', 2601),
 ('complex', 2581),
 ('chickpea', 2520),
 ('mct oil', 2461),
 ('oat milk', 2387),
 ('gluten free', 2330),
 ('sugar free', 2330),
 ('k', 2328),
 ('pill', 2282)]

In [114]:
# all_known_phrases_lst = []
# for file in ['quad_grams.csv', 'bi_grams.csv', 'uni_grams.csv', 'tri_grams.csv', 'penta_grams.csv']:
#     curr_df = pd.read_csv('../resources/' + file)
#     curr_phrases = curr_df.key.values
#     all_known_phrases_lst += list(curr_phrases)

In [115]:
# len(all_known_phrases_lst)

In [49]:
key = [pair[0] for pair in sorted_items]
gram_type = [len(nltk.word_tokenize(pair[0])) for pair in sorted_items]
count = [pair[1] for pair in sorted_items]

additional_phrases = pd.DataFrame({
    'key': key,
    'gram_type': gram_type,
    'count': count
}).sort_values(by = 'gram_type', ascending = False)

additional_phrases.to_csv('../data/result/additional_phrases.csv', )

In [36]:
key[:3], gram_type[:3], count[:3]

(['ml', 'herbal', 'b'], [1, 1, 1], [12170, 8492, 7861])

## Refinment

In [51]:
curr_phrase_bank = [pair[0] for pair in sorted_items]
curr_phrase_bank[:3], len(curr_phrase_bank)

(['ml', 'herbal', 'b'], 1731)

In [52]:
print('=> Length of curr_phrase_bank:', len(curr_phrase_bank))
    
tmp_mask = np.array([True] * len(curr_phrase_bank))
curr_phrase_bank = np.array(curr_phrase_bank)
    
curr_idx = 0
for ph in tqdm(curr_phrase_bank):
    tokens = nltk.word_tokenize(ph)
    curr_gram_length = len(tokens)
    for other_ph in curr_phrase_bank:
        if ph == other_ph:
            continue
            
        other_ph_tokens = nltk.word_tokenize(other_ph)
        if len(other_ph_tokens) < curr_gram_length:
            continue
            
        n_grams = [' '.join(gram) for gram in nltk.ngrams(other_ph_tokens, curr_gram_length)]
        if ph in n_grams:
            tmp_mask[curr_idx] = False
            break
    curr_idx += 1
if sum(tmp_mask) == len(tmp_mask):
    print('    => No Change to current phrase bank!')
#     break
curr_phrase_bank = curr_phrase_bank[tmp_mask]
print(len(curr_phrase_bank))
# break

  0%|          | 4/1731 [00:00<01:04, 26.85it/s]

=> Length of curr_phrase_bank: 1731


100%|██████████| 1731/1731 [03:24<00:00,  8.47it/s]


SyntaxError: 'break' outside loop (<ipython-input-52-2270ebb5ba5d>, line 25)

In [54]:
curr_phrase_bank = curr_phrase_bank[tmp_mask]

print(len(curr_phrase_bank))

1413


In [50]:
1 - 0.189122

0.810878